In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np


class SampaddingConv1D_BN(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size):
        super(SampaddingConv1D_BN, self).__init__()
        self.padding = nn.ConstantPad1d((int((kernel_size-1)/2), int(kernel_size/2)), 0)
        self.conv1d = torch.nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size)
        self.bn = nn.BatchNorm1d(num_features=out_channels)
        
    def forward(self, X):
        X = self.padding(X)
        X = self.conv1d(X)
        X = self.bn(X)
        return X
    
class build_layer_with_layer_parameter_for_loop(nn.Module):
    def __init__(self,layer_parameters): 
        super(build_layer_with_layer_parameter_for_loop, self).__init__()
        self.conv_list = nn.ModuleList()
#############################################################################        
        for i in layer_parameters:
            conv = SampaddingConv1D_BN(i[0],i[1],i[2])
            self.conv_list.append(conv)
#############################################################################       
    def forward(self, X):
        
        conv_result_list = []
        for conv in self.conv_list:
            conv_result = conv(X)
            conv_result_list.append(conv_result)
            
        result = F.relu(torch.cat(tuple(conv_result_list), 1))
        return result

In [2]:
#############################################################################  
def calculate_mask_index(kernel_length_now,largest_kernel_lenght):
    right_zero_mast_length = math.ceil((largest_kernel_lenght-1)/2)-math.ceil((kernel_length_now-1)/2)
    left_zero_mask_length = largest_kernel_lenght - kernel_length_now - right_zero_mast_length
    return left_zero_mask_length, left_zero_mask_length+ kernel_length_now
#############################################################################  
def creat_mask(number_of_input_channel,number_of_output_channel, kernel_length_now, largest_kernel_lenght):
    ind_left, ind_right= calculate_mask_index(kernel_length_now,largest_kernel_lenght)
    mask = np.ones((number_of_input_channel,number_of_output_channel,largest_kernel_lenght))
    mask[:,:,0:ind_left]=0
    mask[:,:,ind_right:]=0
    return mask


def creak_layer_mask(layer_parameter_list):
    largest_kernel_lenght = layer_parameter_list[-1][-1]
    mask_list = []
    init_weight_list = []
    bias_list = []
    for i in layer_parameter_list:
        conv = torch.nn.Conv1d(in_channels=i[0], out_channels=i[1], kernel_size=i[2])
        ind_l,ind_r= calculate_mask_index(i[2],largest_kernel_lenght)
        big_weight = np.zeros((i[1],i[0],largest_kernel_lenght))
        big_weight[:,:,ind_l:ind_r]= conv.weight.detach().numpy()
        
        bias_list.append(conv.bias.detach().numpy())
        init_weight_list.append(big_weight)
        
        mask = creat_mask(i[1],i[0],i[2], largest_kernel_lenght)
        mask_list.append(mask)
        
    mask = np.concatenate(mask_list, axis=0)
    init_weight = np.concatenate(init_weight_list, axis=0)
    init_bias = np.concatenate(bias_list, axis=0)
    return mask.astype(np.float32), init_weight.astype(np.float32), init_bias.astype(np.float32)

    
class build_layer_with_layer_parameter_mask(nn.Module):
    def __init__(self,layer_parameters):
        super(build_layer_with_layer_parameter_mask, self).__init__()

        os_mask, init_weight, init_bias= creak_layer_mask(layer_parameters)
        
        
        in_channels = os_mask.shape[1] 
        out_channels = os_mask.shape[0] 
        max_kernel_size = os_mask.shape[-1]

        self.weight_mask = nn.Parameter(torch.from_numpy(os_mask),requires_grad=False)
        
        self.padding = nn.ConstantPad1d((int((max_kernel_size-1)/2), int(max_kernel_size/2)), 0)
         
        self.conv1d = torch.nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=max_kernel_size)
        self.conv1d.weight = nn.Parameter(torch.from_numpy(init_weight),requires_grad=True)
        self.conv1d.bias =  nn.Parameter(torch.from_numpy(init_bias),requires_grad=True)

        self.bn = nn.BatchNorm1d(num_features=out_channels)
    
    def forward(self, X):
        self.conv1d.weight.data = self.conv1d.weight*self.weight_mask
        #self.conv1d.weight.data.mul_(self.weight_mask)
        result_1 = self.padding(X)
        result_2 = self.conv1d(result_1)
        result_3 = self.bn(result_2)
        result = F.relu(result_3)
        return result    

In [3]:
layer_parameters = [(1, 10, 1), (1, 10, 2), (1, 10, 3), (1, 10, 5), (1, 10, 7), (1, 10, 11), (1, 10, 13), (1, 10, 17), (1, 10, 19), (1, 10, 23)]

In [4]:
import time

layer_for= build_layer_with_layer_parameter_for_loop(layer_parameters)
input_tensor = torch.randn(10, 1, 200)

start = time.time()
for i in range(100):
    temp = layer_for(input_tensor)
end = time.time()
print(end-start)

0.2979147434234619


In [5]:
import time

layer_mask= build_layer_with_layer_parameter_mask(layer_parameters)
input_tensor = torch.randn(10, 1, 200)

start = time.time()
for i in range(100):
    temp = layer_mask(input_tensor)
end = time.time()
print(end-start)

0.09134984016418457


In [6]:
layer_for= build_layer_with_layer_parameter_for_loop(layer_parameters)
print(next(layer_for.parameters()).is_cuda)
layer_for = layer_for.to('cuda:0')
print(next(layer_for.parameters()).is_cuda)

input_tensor = torch.randn(5, 1, 200)
input_tensor = input_tensor.to('cuda:0')


start = time.time()
for i in range(100):
    temp = layer_for(input_tensor)
end = time.time()
print(end-start)

False
True
0.3223726749420166


In [7]:
layer_mask= build_layer_with_layer_parameter_mask(layer_parameters)
print(next(layer_mask.parameters()).is_cuda)
layer_mask = layer_mask.to('cuda:0')
print(next(layer_mask.parameters()).is_cuda)

input_tensor = torch.randn(5, 1, 200)
input_tensor = input_tensor.to('cuda:0')


start = time.time()
for i in range(100):
    temp = layer_mask(input_tensor)
end = time.time()
print(end-start)

False
True
0.024903297424316406
